In [1]:
from datetime import datetime

def datetimeFromUnixTime(t):
    ts = int(t)
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

def getCurrentTime():
    return str(datetime.now())

print(datetimeFromUnixTime(1557397225))
print(getCurrentTime())

2019-05-09 10:20:25
2019-05-12 00:38:30.650481


In [1]:
import mysql.connector

engine = mysql.connector.connect(
    user="ecohydro",
    password="7fQh53G6D2BuCnT54hXH95CJx",
    host="ubc-ecohydro-nodes-mysql-database.cycweraudmq1.us-east-1.rds.amazonaws.com",
    port='3306',
    database='eco_nodes'
)

#insertDataQuery = 'datacall eco_nodes.insert_data(%s, {boot_count}, {sensor}, {param}, {read_time}, {store_time}, {reading});


data = "02&1557397229&0&2:15266.1,21.7&3:100000.0,21.2&17:1.41,21.5,1&22:1.76,21.4,1"


def writeDataToDatabase(node, boot_count, sensor, param, read_time, store_time, reading):
    #cur.execute(f'CREATE TABLE {node} (id serial PRIMARY KEY, num float, data varchar);')
    print(f'call eco_nodes.insert_data({node}, {boot_count}, {sensor}, {param}, \'{read_time}\', \'{store_time}\', {reading});')
    cur.execute(f'call eco_nodes.insert_data({node}, {boot_count}, {sensor}, {param}, \'{read_time}\', \'{store_time}\', {reading});')
    #cur.execute("SELECT * FROM test;")
    #cur.execute(insertDataQuery, values)
    engine.commit()


def parseDataString(string):
    entries = string.split('&')
        
    node = entries[0]
    read_time = datetimeFromUnixTime(entries[1])
    store_time = getCurrentTime()
    
    boot_count = entries[2]
    
    for entry in entries[3:]:
        e = entry.split(':')
        
        sensor = e[0]
        readings = e[1].split(',')
        
        for param, reading in enumerate(readings):
            #writeDataToDatabase(int(node), int(boot_count), int(sensor), param, read_time, store_time, float(reading));
            writeDataToDatabase(node, boot_count, sensor, param, read_time, store_time, reading);

parseDataString(data)

cur.close()
engine.close()

NameError: name 'datetimeFromUnixTime' is not defined

In [1]:

from http.server import BaseHTTPRequestHandler
#from socketserver import ThreadingMixIn
#import threading
import cgi

import mysql.connector


class PostHandler(BaseHTTPRequestHandler):
    
    # Initialize variables for communicating with data-base
    def __init__(self):
       
        self.engine = mysql.connector.connect(
            user="ecohydro",
            password="7fQh53G6D2BuCnT54hXH95CJx",
            host="ubc-ecohydro-nodes-mysql-database.cycweraudmq1.us-east-1.rds.amazonaws.com",
            port='3306',
            database='eco_nodes'
        )
        
        self.cur = engine.cursor()

    
    def do_POST(self):
        # Parse the form data posted
        form = cgi.FieldStorage(
            fp=self.rfile, 
            headers=self.headers,
            environ={'REQUEST_METHOD':'POST',
                     'CONTENT_TYPE':self.headers['Content-Type'],
                     })

        # Echo back information about what was posted in the form
        for field in form.keys():
            field_item = form[field]
            # Regular form value
            print('\t%s=%s\n' % (field, form[field].value))
        return
        
        
      #  if string[0] == 't':
      #      parseTitleString(string)
      #  else:
      #      parseDataString(string)
        
    
    def writeDataToDatabase(node, boot_count, sensor, param, read_time, store_time, reading):
        print(f'call eco_nodes.insert_data({node}, {boot_count}, {sensor}, {param}, \'{read_time}\', \'{store_time}\', {reading});')
        self.cur.execute(f'call eco_nodes.insert_data({node}, {boot_count}, {sensor}, {param}, \'{read_time}\', \'{store_time}\', {reading});')
        self.engine.commit()
    
    def parseDataString(string):
        entries = string.split('&')

        node = entries[0]
        read_time = datetimeFromUnixTime(entries[1])
        store_time = getCurrentTime()

        boot_count = entries[2]

        for entry in entries[3:]:
            e = entry.split(':')

            sensor = e[0]
            readings = e[1].split(',')

            for param, reading in enumerate(readings):
                #writeDataToDatabase(int(node), int(boot_count), int(sensor), param, read_time, store_time, float(reading));
                writeDataToDatabase(node, boot_count, sensor, param, read_time, store_time, reading);
    
    def parseTitleString(string):
        print("title string: " + string)
    
    # Cleanup variables for talking to database
    def __del__(self):
        this.cur.close()
        this.engine.close()
    
    
if __name__ == '__main__':
    from http.server import HTTPServer
    server = HTTPServer(('0.0.0.0', 5200), PostHandler)
    print('Starting server, use <Ctrl-C> to stop')
    server.serve_forever()

NameError: name 'BaseHTTPRequestHandler' is not defined

In [ ]:
# PUSH server refference https://pymotw.com/2/BaseHTTPServer/index.html#module-BaseHTTPServer

from http.server import BaseHTTPRequestHandler
from socketserver import ThreadingMixIn
import threading
from cgi import parse_header, parse_multipart
import json

import datetime
from time import gmtime, strftime
import time

import pymysql.cursors

# wrapper for communicating with database
class EcoDatabase:

    # Initialize variables for communicating with data-base
    def __init__(self):
        print("starting database")
        self.connection = pymysql.connect(host='ubc-ecohydro-nodes-mysql-database.cycweraudmq1.us-east-1.rds.amazonaws.com',
                                          port=3306,
                                          user='ecohydro',
                                          password='7fQh53G6D2BuCnT54hXH95CJx',
                                          database='eco_nodes',
                                          charset='utf8mb4',
                                          cursorclass=pymysql.cursors.DictCursor)
        self.cursor = self.connection.cursor()
        print("database started")


    # Cleanup variables for talking to database
    def __del__(self):
        print("closed")
        self.cursor.close()
        self.connection.close()


    def insert_data_point(self, node_id, boot_count, sensor_address, parameter_number, read_time, store_time, data_point):
        query = f'call eco_nodes.insert_data_point({node_id}, {boot_count}, {sensor_address}, {parameter_number}, \'{read_time}\', \'{store_time}\', {data_point});'
        print(query)
        self.cursor.execute(query)
        self.connection.commit()

    def insert_node_sensor(self, node_id, boot_count, sensor_address, sensor_serial_number):
        query = f'call eco_nodes.insert_node_sensor({node_id}, {boot_count}, {sensor_address}, {sensor_serial_number});'
        print(query)
        self.cursor.execute(query)
        self.connection.commit()

    def insert_node_setup(self, node_id, boot_count, boot_time, store_time, position="", comment=""):
        query = f'call eco_nodes.insert_node_setup({node_id}, {boot_count}, \'{boot_time}\', \'{store_time}\', {position}, {comment});'
        print(query)
        self.cursor.execute(query)
        self.connection.commit()

    def insert_sensor_type(self, id, type, name, manufacturer, description):
        query = f'call eco_nodes.insert_sensor_type({id}, \'{type}\', \'{name}\', \'{manufacturer}\', \'{description}\');'
        print(query)
        self.cursor.execute(query)
        self.connection.commit()


        
class PostHandler(BaseHTTPRequestHandler):

    database = EcoDatabase()
    
    def do_POST(self):
        # Parse the form data posted

        data_string = self.rfile.read(int(self.headers['Content-Length']))
        data = json.loads(data_string)
        print(data)

        if (not data['app_id'] == 'ubc_econode_test_network'):
            return # message is not from a node

        nodeString = data['dev_id']
        payload = data['payload_fields']['data']
        print('Parsing ', payload)

        # authentication

        # extract node information
        node_id = ''.join([s for s in nodeString if s.isdigit()])
        print(node_id)
        print(type(node_id))


        if payload[0] == 'C':
            # config message
            self.parseTitleString(node_id, payload)
        else:
            # data message
            self.parseDataString(node_id, payload)

        return

    def parseDataString(self, node_id, string):
        entries = string.split('&')

        boot_count = entries[0]

        read_time = datetime.datetime.fromtimestamp(int(entries[1])).strftime('%Y-%m-%d %H:%M:%S')
        store_time = strftime("%Y-%m-%d %H:%M:%S", gmtime())

        for entry in entries[2:]:
            e = entry.split(':')

            sensor_address = e[0]
            readings = e[1].split(',')

            for parameter_number, data_point in enumerate(readings):
                #writeDataToDatabase(int(node), int(boot_count), int(sensor), param, read_time, store_time, float(reading))
                self.database.insert_data_point(node_id, boot_count, sensor_address, parameter_number, read_time, store_time, data_point)

    def parseTitleString(self, node_id, string):
        entries = string[1:].split('&')

        boot_count = entries[0]

        boot_time = datetime.datetime.fromtimestamp(int(entries[1])).strftime('%Y-%m-%d %H:%M:%S')
        store_time = strftime("%Y-%m-%d %H:%M:%S", gmtime())

        self.database.insert_node_setup(node_id, boot_count, boot_time, store_time)

        for entry in entries[2:]:
            e = entry.split(':')

            sensor_address = e[0]
            sensor_serial_number = e[1]

            self.database.insert_node_sensor(node_id, boot_count, sensor_address, sensor_serial_number)


        
if __name__ == '__main__':
    from http.server import HTTPServer
    server = HTTPServer(('0.0.0.0', 5200), PostHandler)
    print('Starting server, use <Ctrl-C> to stop')
    server.serve_forever()


starting database
database started
Starting server, use <Ctrl-C> to stop


In [2]:
import pymysql.cursors

connection = pymysql.connect(host='localhost',
                             user='user',
                             password='passwd',
                             database='db',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

try:
    with connection.cursor() as cursor:
        # Create a new record
        sql = "INSERT INTO `users` (`email`, `password`) VALUES (%s, %s)"
        cursor.execute(sql, ('webmaster@python.org', 'very-secret'))

    # connection is not autocommit by default. So you must commit to save
    # your changes.
    connection.commit()

    with connection.cursor() as cursor:
        # Read a single record
        sql = "SELECT `id`, `password` FROM `users` WHERE `email`=%s"
        cursor.execute(sql, ('webmaster@python.org',))
        result = cursor.fetchone()
        print(result)
finally:
    connection.close()

OperationalError: (1045, "Access denied for user 'user'@'localhost' (using password: YES)")

In [ ]:


from flask import Flask, request

app = Flask(__name__)

@app.route("/", methods=['POST', 'GET'])
def handleData():
        if request.method == 'POST':
                jsonData = request.get_json()
                print(jsonData['payload_fields']['data'])
                return '...'
        return "Running and ready"


if __name__ == '__main__':
        app.run(debug=True, host='0.0.0.0', port=5200)

